In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [28]:
# Load the dataset
# df = pd.read_csv('all_features.csv')
# df = pd.read_csv('all_features.csv', low_memory=False)
df = pd.read_csv("all_features.csv", engine='python', error_bad_lines=False)


# df = df.dropna()
df.shape
# df.head()


(1180, 152)

In [32]:
df['exercise'].value_counts()


burpees          236
crunches         236
squats           236
jumping_jacks    236
plank            236
Name: exercise, dtype: int64

In [ ]:
print("Number of unique exercises:", df['exercise'].nunique())
print("Unique exercises:", df['exercise'].unique())
# count nans
df.isna().sum()
df.dropna(subset=['exercise'], inplace=True)

Number of unique exercises: 5
Unique exercises: ['burpees' 'crunches' 'jumping_jacks' 'plank' 'squats']


acc_x_mean              0
acc_x_std               0
acc_x_min               0
acc_x_max               0
acc_x_range             0
                       ..
exercise_participant    0
window_id               0
start_time              0
end_time                0
duration                0
Length: 152, dtype: int64

In [ ]:
# convert 'exercise' to categorical
df['exercise'] = df['exercise'].astype('category')
# convert 'exercise' to numerical codes
df['exercise_code'] = df['exercise'].cat.codes
# check the unique codes
print("Unique exercise codes:", df['exercise_code'].unique())
# code mapping
# exercise_mapping = dict(zip(df['exercise'].cat.categories, df['exercise'].cat.codes))
# df['exercise_code'] = df['exercise'].map(exercise_mapping)
# check the unique codes
# print("Unique exercise codes:", df['exercise_code'].unique())

df.head()


Unique exercise codes: [0 1 2 3 4]
Unique exercise codes: [0 1 2 3 4]


,acc_x_mean,acc_x_std,acc_x_min,acc_x_max,acc_x_range,acc_x_median,acc_x_q25,acc_x_q75,acc_x_iqr,acc_x_energy,...,mag_magnitude_max,exercise,participant,dataset,exercise_participant,window_id,start_time,end_time,duration,exercise_code
0,-5.268432,1.904335,-8.247998,-2.079326,6.168672,-5.060676,-7.221321,-3.493510,3.727810,31.382871,...,45.875848,burpees,akshaye,burpees1,burpees_akshaye,0,0.001587,0.987596,0.986009,0
1,-3.654246,0.970385,-5.878725,-1.686393,4.192332,-3.688555,-4.445830,-2.825113,1.620717,14.295160,...,46.187853,burpees,akshaye,burpees1,burpees_akshaye,1,0.499570,1.485582,0.986011,0
2,-3.513426,2.537133,-5.908663,7.667356,13.576019,-4.312607,-4.863985,-3.077712,1.786273,18.781207,...,46.187853,burpees,akshaye,burpees1,burpees_akshaye,2,0.997556,1.983567,0.986010,0
3,-1.826836,3.515869,-5.908663,7.667356,13.576019,-2.703079,-4.847108,0.804764,5.651872,15.698666,...,46.085559,burpees,akshaye,burpees1,burpees_akshaye,3,1.495542,2.481551,0.986010,0
4,-0.713563,2.688073,-7.833959,5.494474,13.328433,-0.306413,-2.357747,0.837097,3.194844,7.734911,...,46.085559,burpees,akshaye,burpees1,burpees_akshaye,4,1.993527,2.979537,0.986011,0


In [ ]:
columns_to_drop = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'exercise', 'participant' , 'dataset' , 'exercise_participant', 'gyro_x_mean', 'gyro_x_std', 'gyro_x_min', 'gyro_x_max', 'gyro_x_range',
       'gyro_x_median', 'gyro_x_q25', 'gyro_x_q75', 'gyro_x_iqr',
       'gyro_x_energy', 'gyro_x_rms', 'gyro_magnitude_mean',
       'gyro_magnitude_std', 'gyro_magnitude_max']
df.drop([col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)
# Check for NaN values again
print("NaN values after dropping columns:", df.isna().sum().sum())

NaN values after dropping columns: 0


In [65]:
print(df.columns[df.isna().any()])


Index([], dtype='object')


In [68]:

# Compute correlation matrix (excluding non-numeric columns automatically)
# df_numeric = df.select_dtypes(include=['number'])
# corr = df_numeric.corr()
corr = df.corr()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .75})
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\skhur\miniconda3\envs\mlqs\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\skhur\AppData\Local\Temp\ipykernel_11092\1717613252.py", line 8, in <module>
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .75})
  File "c:\Users\skhur\miniconda3\envs\mlqs\lib\site-packages\seaborn\matrix.py", line 446, in heatmap
    plotter = _HeatMapper(data, vmin, vmax, cmap, center, robust, annot, fmt,
  File "c:\Users\skhur\miniconda3\envs\mlqs\lib\site-packages\seaborn\matrix.py", line 113, in __init__
    mask = _matrix_mask(data, mask)
  File "c:\Users\skhur\miniconda3\envs\mlqs\lib\site-packages\seaborn\matrix.py", line 92, in _matrix_mask
    mask = mask | pd.isnull(data)
  File "c:\Users\skhur\miniconda3\envs\mlqs\lib\site-packages\pandas\core\ops\__init__.py", line 767, in f
  File "c:\Users\skhur\mini

<Figure size 1200x800 with 0 Axes>

In [72]:
# Define features and labels
columns_to_drop=['exercise', 'participant' , 'dataset' , 'exercise_participant', 'gyro_x_mean', 'gyro_x_std', 'gyro_x_min', 'gyro_x_max', 'gyro_x_range',
       'gyro_x_median', 'gyro_x_q25', 'gyro_x_q75', 'gyro_x_iqr',
       'gyro_x_energy', 'gyro_x_rms', 'gyro_magnitude_mean',
       'gyro_magnitude_std', 'gyro_magnitude_max', 'exercise_code']
existing_cols_to_drop = [col for col in columns_to_drop if col in df.columns]
X = df.drop(columns=existing_cols_to_drop, errors='ignore')

y = df['exercise_code']


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23, stratify=y)


# # Drop non-numeric or irrelevant columns
# # columns_to_drop = ['exercise', 'participant', 'dataset', 'exercise_participant', 'window_id', 'start_time', 'end_time', 'duration']
# columns_to_drop = ['exercise', 'participant' , 'dataset' , 'exercise_participant', 'gyro_x_mean', 'gyro_x_std', 'gyro_x_min', 'gyro_x_max', 'gyro_x_range',
#        'gyro_x_median', 'gyro_x_q25', 'gyro_x_q75', 'gyro_x_iqr',
#        'gyro_x_energy', 'gyro_x_rms', 'gyro_magnitude_mean',
#        'gyro_magnitude_std', 'gyro_magnitude_max']
# X = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# # Make sure all feature columns are numeric
# X = X.apply(pd.to_numeric, errors='coerce')


# nan_counts = X.isna().sum()
# print(nan_counts[nan_counts > 0])


# # Check for NaNs now
# print("NaNs before split:", X.isna().sum().sum())

# # Drop rows with NaNs before train/test split
# # X = X.dropna()
# y = df.loc[X.index, 'exercise']  # Re-align y with cleaned X

# # Split the data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=23)


In [38]:
print("NaNs in X_train:", X_train.isna().sum().sum())
print("Infs in X_train:", np.isinf(X_train).sum().sum())


NaNs in X_train: 10024
Infs in X_train: 0


In [13]:
print(X_train.dtypes)


acc_x_mean                  float64
acc_x_std                   float64
acc_x_min                   float64
acc_x_max                   float64
acc_x_range                 float64
                             ...   
linear_acc_magnitude_std    float64
linear_acc_magnitude_max    float64
mag_magnitude_mean          float64
mag_magnitude_std           float64
mag_magnitude_max           float64
Length: 144, dtype: object


In [73]:
# Random Forest classification function
def random_forest_classification(X_train, y_train, X_test, y_test, n_estimators=100, max_depth=10):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=23)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, predictions)
    return accuracy

In [74]:
# KNN classification function
def knn_classification(X_train, y_train, X_test, y_test, n_neighbors=5):
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, predictions)
    return accuracy

In [75]:
# Run models and print results
rf_acc = random_forest_classification(X_train, y_train, X_test, y_test)
knn_acc = knn_classification(X_train, y_train, X_test, y_test)

print(f'Random Forest Accuracy: {rf_acc:.4f}')
print(f'KNN Accuracy: {knn_acc:.4f}')

Random Forest Accuracy: 0.9788
KNN Accuracy: 0.9322
